In [1]:
import polars as pl;
import csv;

Carga de datos desde Drive

In [2]:

from google.colab import drive, files

drive.mount('/content/drive')
log_file_path = '/content/drive/My Drive/Colab_Folder/access.log'
csv_file_path = '/content/drive/My Drive/Colab_Folder/archivo.csv'

Mounted at /content/drive


Convierto el archivo .logs a .csv para poder trabajarlo con Polars

In [3]:
# Abro el archivo original .log
with open(log_file_path, 'r') as file:
    log_data = file.readlines()

# Los encabezados para el archivo .csv
headers = ["ip", "timestamp", "request_method", "url", "http_version", "status_code", "size", "user_agent"]

# Lo leo linea por linea y voy guardando la info en el archivo.csv
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)

    for line in log_data:
        parts = line.split(' ')

        try:
            ip = parts[0]
            timestamp = ' '.join(parts[3:5]).strip('[]')
            request_method = parts[5].strip('"') if parts[5] != '"-"' else None
            url = parts[6] if len(parts) > 6 else None
            http_version = parts[7].strip('"') if len(parts) > 7 else None
            status_code = int(parts[8]) if len(parts) > 8 and parts[8].isdigit() else None
            size = int(parts[9]) if len(parts) > 9 and parts[9].isdigit() else None
            user_agent = ' '.join(parts[11:]).strip('"') if len(parts) > 11 else None

            csvwriter.writerow([ip, timestamp, request_method, url, http_version, status_code, size, user_agent])

        except (IndexError, ValueError) as e:
            print(f"Error procesando la línea: {line.strip()} - Error: {e}")
            csvwriter.writerow([ip, timestamp, None, None, None, None, None, None])

### Funciones Auxiliares

#### **filtrar_map_server:**
MapServer es una plataforma de código abierto para publicar datos espaciales y aplicaciones interactivas de mapeo en la web.
libcurl es una biblioteca utilizada por MapServer para manejar transferencias de datos a través de varios protocolos de red.

---

#### **filtar_wms:**
Un WMS (Web Map Service) es un estándar desarrollado por el OGC (Open Geospatial Consortium) que permite la visualización de mapas georreferenciados a través de la web. Un servicio WMS puede devolver imágenes de mapas basadas en datos geográficos, y el documento de capacidades describe cómo se pueden solicitar estas imágenes y qué capas están disponibles.

Dado que todas las solicitudes son muy cercanas en tiempo y provienen de la misma ip, se podría decir que: Un usuario hizo varias solicitudes consecutivas rápidamente. O, lo que es más probable, un script o un programa hizo esas solicitudes.

---

#### **filtrar_jcemediabox:**
Estas URLs están asociadas con JCEMediaBox, un plugin de Joomla que facilita la incorporación de lightboxes para mostrar imágenes, vídeos y otros tipos de medios de una manera atractiva y funcional. Los archivos .js contienen el código JavaScript necesario para hacer funcionar el lightbox, mientras que los archivos .css contienen los estilos que determinan la apariencia del lightbox en la página web.

In [28]:
def filtrar_googlebot(df):
    return df.filter(~pl.col("user_agent").str.contains('Googlebot'))

def filtrar_baiduspider(df):
    return df.filter(~pl.col("user_agent").str.contains('Baiduspider'))

def filtrar_agesic_crawler(df):
    return df.filter(~pl.col("user_agent").str.contains('agesic-crawler'))

def filtrar_robots_y_crawlers(df):
    df = filtrar_googlebot(df)
    df = filtrar_baiduspider(df)
    df = filtrar_agesic_crawler(df)
    return df

def filtrar_map_server(df):
    return df.filter(~pl.col("url").str.contains('/wfsPCN1000.cgi'))

def filtrar_wms(df):
    return df.filter(~pl.col("url").str.contains('SERVICE=WMS'))

def filtrar_solicitudes_servicios_mapas(df):
    df = filtrar_map_server(df)
    df = filtrar_wms(df)
    return df

def filtar_jcemediabox(df):
    return df.filter(~pl.col("url").str.contains('/plugins/system/jcemediabox/'))

def filtrar_datos(df):
    df = filtrar_robots_y_crawlers(df)
    df = filtrar_solicitudes_servicios_mapas(df)
    df = filtar_jcemediabox(df)
    return df

def filtrar_URLs_vacias(df):
    return df.filter(pl.col("url").is_not_null())

def count_frecuencia_url(df):
    return df.group_by("url").agg(pl.col("url").count().alias("visit_count"))

def formatear_fecha(df):
    return df.with_columns([pl.col("timestamp").str.strptime(pl.Datetime, "%d/%b/%Y:%H:%M:%S %z", strict=False).alias("timestamp")])


### Main

In [33]:
df = pl.read_csv(csv_file_path)

# Limpieza de datos
df_filtrado = filtrar_datos(df)
df_cleaned = formatear_fecha(df_filtrado)
#print(df_cleaned) # .glimpse() para ver string completo

# Páginas más visitadas
df_frecuencia_url_ordenado = count_frecuencia_url(filtrar_URLs_vacias(df_cleaned)).sort("visit_count", descending=True)
df_frecuencia_url_ordenado.head(10).glimpse()
#df_frecuencia_url_ordenado.top_k(10, by="visit_count")

Rows: 10
Columns: 2
$ url         <str> '*', '/', '/map.phtml', '/templates/system/css/general.css', '/templates/system/css/system.css', '/media/system/js/mootools-more.js', '/plugins/system/jat3/jat3/base-themes/default/css/typo.css', '/plugins/system/jat3/jat3/base-themes/default/css/addons.css', '/templates/bt_arise/css/typo.css', '/templates/bt_arise/js/jquery-1.6.2.min.js'
$ visit_count <u32> 5386, 2822, 841, 652, 651, 650, 649, 648, 647, 642

